# 아마존 세이지메이커 점프스타트에서 Llama 2 모델 실행

In [5]:
model_id = "meta-textgeneration-llama-2-7b-f"

In [9]:
model_version = "2.*"

## 모델 배포

***
이제 세이지메이커 점프스타트를 활용해 모델을 배포할 수 있습니다. 배포가 성공하려면 모델의 배포 메서드에서 `accept_eula` 인수를 수동으로 `True`로 변경해야 합니다.
***

In [10]:
from sagemaker.jumpstart.model import JumpStartModel

model = JumpStartModel(model_id=model_id, model_version=model_version)

For forward compatibility, pin to model_version='2.*' in your JumpStartModel or JumpStartEstimator definitions. Note that major version upgrades may have different EULA acceptance terms and input/output signatures.
Using model 'meta-textgeneration-llama-2-7b-f' with wildcard version identifier '2.*'. You can pin to version '2.0.4' for more stable results. Note that models may have different input/output signatures after a major version upgrade.


In [11]:
predictor = model.deploy(accept_eula=True)

-----------------!

## 엔드포인트 호출

***
### 지원되는 매개변수

***
이 모델은 추론 수행 중 다음과 같은 여러 매개변수를 지원합니다.

* **max_length:** 모델은 출력 길이(입력 콘텍스트 포함)가 `max_length`에 도달할 때까지 텍스트를 생성합니다. 지정 시 양의 정수여야 합니다.
* **max_new_tokens:** 모델이 출력 길이(입력 콘텍스트 길이 제외)가 `max_new_tokens`에 도달할 때까지 텍스트를 생성합니다. 지정 시 양의 정수여야 합니다.
* **num_beams:** 탐욕적 탐색에 사용되는 빔 수입니다. 지정 시 `num_return_sequences`이상의 정수여야 합니다.
* **no_repeat_ngram_size:** 모델이 `no_repeat_ngram_size`의 단어 시퀀스가 출력 시퀀스에서 반복되지 않도록 합니다. 지정 시 1보다 큰 양의 정수여야 합니다.
* **temperature:** 출력의 무작위성을 제어합니다. 높은 온도는 확률이 낮은 단어로 구성된 출력 시퀀스를 생성하며, 낮은 온도는 확률이 높은 단어로 구성된 출력 시퀀스를 생성합니다. `temperature`가 0이면 탐욕적 디코딩이 됩니다. 지정 시 양의 실수여야 합니다.
* **early_stopping:** True일 경우, 모든 빔 가설이 문장 종료 토큰에 도달하면 텍스트 생성이 종료됩니다. 지정 시 boolean이어야 합니다.
* **do_sample:** True일 경우, 다음 단어를 확률에 따라 샘플링합니다. 지정 시 boolean이어야 합니다.
* **top_k:** 텍스트 생성의 각 단계에서, 가장 가능성이 높은 `top_k`개의 단어에서만 샘플링합니다. 지정 시 양의 정수여야 합니다.
* **top_p:** 텍스트 생성의 각 단계에서, 누적 확률이 `top_p`인 단어의 최소 집합에서 샘플링합니다. 지정 시 0과 1 사이의 실수여야 합니다.
* **return_full_text:** True일 경우, 입력 텍스트가 생성된 텍스트의 일부로 포함됩니다. 지정 시 boolean이어야 합니다. 기본값은 False입니다.
* **stop**: 지정하는 경우, 문자열 목록이어야 합니다. 지정된 문자열 중 하나가 생성되면 텍스트 생성이 멈춥니다.

엔드포인트 호출 시, 위 매개변수 중 일부를 지정할 수 있습니다. 아래는 이러한 인수를 활용한 호출 예시입니다.

**참고**: `max_new_tokens`가 정의되지 않은 경우, 모델은 최대 총 토큰 수인 4K까지 생성할 수 있습니다. 이는 엔드포인트 쿼리 타임아웃 오류를 유발할 수 있으므로 가능한 경우 `max_new_tokens`를 설정하는 것이 좋습니다. 7B, 13B 및 70B 모델의 경우, 총 토큰 수를 4K 이하로 유지하면서 각각 1500, 1000, 500을 초과하지 않도록 `max_new_tokens`를 설정하는 것이 좋습니다.

**참고**: 이 모델은 'system', 'user', 'assistant' 역할만 지원하며, 'system'으로 시작해 'user'와 번갈아 가며 (system/user/assistant/user/assistant/user...) 진행됩니다.
***

### 프롬프트 예시
***
이 섹션의 예시는 대화형 다이얼로그를 프롬프트 입력으로 텍스트를 생성하는 방법을 보여줍니다. 페이로드 예시는 `JumpStartModel` 객체에서 프로그래밍 방식으로 검색됩니다.

Llama-2 채팅 모델의 입력 메시지는 다음 형식을 따라야 합니다. 이 모델은 ‘system’, ‘user’, ‘assistant’ 역할만 지원하며, ‘system’으로 시작해 ‘user’와 번갈아 가며 진행됩니다 (system/user/assistant/user/assistant/user…). 마지막 메시지는 반드시 ‘user’여야 합니다. 간단한 사용자 프롬프트 예시는 다음과 같습니다.

```
<s>[INST] {user_prompt} [/INST]
```

다음 구문을 사용해 시스템 프롬프트를 추가할 수 있습니다.

```
<s>[INST] <<SYS>>
{system_prompt}
<</SYS>>

{user_prompt} [/INST]

```

마지막으로 이전의 모든 사용자 프롬프트와 어시스턴트 응답을 입력에 포함해 모델과 대화형 상호 작용을 할 수 있습니다.

```
<s>[INST] <<SYS>>
{system_prompt}
<</SYS>>

{user_prompt_1} [/INST] {assistant_response_1} </s><s>[INST] {user_prompt_1} [/INST]
```
***

In [28]:
example_payloads = model.retrieve_all_examples()
example_payloads[0].body

{'inputs': [[{'role': 'user',
    'content': 'what is the recipe of mayonnaise?'}]],
 'parameters': {'max_new_tokens': 512, 'top_p': 0.9, 'temperature': 0.6}}

In [29]:
response = predictor.predict(example_payloads[0].body, custom_attributes="accept_eula=true")
    
print(response[0]["generation"]["content"])

 Mayonnaise is a thick, creamy condiment made from a mixture of egg yolks, oil, and vinegar or lemon juice. Here is a basic recipe for homemade mayonnaise:

Ingredients:

* 2 egg yolks
* 1/2 cup (120 ml) neutral-tasting oil, such as canola or grapeseed
* 1 tablespoon (15 ml) vinegar or lemon juice
* Salt and pepper to taste

Instructions:

1. In a medium-sized bowl, whisk together the egg yolks and salt until they become light in color and frothy.
2. Slowly pour in the oil while continuously whisking the mixture. The mixture will begin to thicken and emulsify as you add the oil.
3. Once you have added about half of the oil, add the vinegar or lemon juice and continue whisking until the mixture is smooth and creamy.
4. Taste and adjust the seasoning as needed. You can add more salt, pepper, or vinegar to taste.
5. Cover the bowl with plastic wrap and chill the mayonnaise in the refrigerator for at least 30 minutes before serving. This will allow the flavors to meld together and the mayo

***
이전 예시 페이로드에는 없지만, 다음 유틸리티 함수로 Llama-2 모델에 메시지를 형식화할 수 있습니다.
***

In [30]:
payload = {
    'inputs': [
        [{
            'role': 'system',
            'content': 'Always answer with Haiku.'
         },
         {
            'role': 'user',
            'content': 'I am going to Paris, what should I see?'
         }]            
    ],
    'parameters': {
        'max_new_tokens': 512, 'top_p': 0.9, 'temperature': 0.6
    }
}

response = predictor.predict(payload, custom_attributes="accept_eula=true")

print(response[0]["generation"]["content"])

 Eiffel Tower high
River Seine flows through city
Art, love, and dreams


## 엔드포인트 정리

In [ ]:
# # 세이지메이커 엔드포인트 삭제
# predictor.delete_model()
# predictor.delete_endpoint()